In [1]:
from mmwave.dataloader import DCA1000
import mmwave.dsp as dsp
from mmwave.tracking import EKF
from mmwave.dsp.doppler_processing import separate_tx
import numpy as np
import utils.configuration as cfg
from datetime import datetime,timedelta
import matplotlib.pyplot as plt
import warnings
from numba import objmode
from mpl_toolkits.mplot3d import Axes3D



In [2]:
import numpy as np

NUM_TX = 3 # tx order tx0,tx2,tx1, face to the board (left,right,upper) 
NUM_RX = 4
VIRT_ANT = NUM_TX*NUM_RX
SKIP_SIZE = 4
START_FREQ = 77 
ADC_START_TIME = 6 
FREQ_SLOPE = 60.012
ADC_SAMPLES = 256 # data samples per chirp
SAMPLE_RATE = 4400
RX_GAIN = 30 
Tp = 14e-6
Tc = 72e-6

IDLE_TIME = 7
RAMP_END_TIME = 65
#  2 for 1843
NUM_FRAMES = 0 
#  Set this to 0 to continuously stream data
LOOPS_PER_FRAME = 182 # num of chirp loop, one loop has three chirps
#PERIODICITY = 100 
# time for one chirp in ms  100ms == 10FPS
ANGLE_RES = 1
ANGLE_RANGE = 90
NUM_DOPPLER_BINS = LOOPS_PER_FRAME
NUM_RANGE_BINS = ADC_SAMPLES
NUM_ANGLE_BINS = 64
RANGE_RESOLUTION = (3e8 * SAMPLE_RATE * 1e3) / (2 * FREQ_SLOPE * 1e12 * ADC_SAMPLES)
MAX_RANGE = (300 * SAMPLE_RATE) / (2 * FREQ_SLOPE * 1e3)
DOPPLER_RESOLUTION = 3e8 / (2 * START_FREQ * 1e9 * (IDLE_TIME + RAMP_END_TIME) * 1e-6 * NUM_DOPPLER_BINS * NUM_TX)
MAX_DOPPLER = 3e8 / (4 * START_FREQ * 1e9 * (IDLE_TIME + RAMP_END_TIME) * 1e-6 * NUM_TX)

MMWAVE_RADAR_LOC=np.array([[0.146517, -3.030810, 1.0371905]]) # hard code the location of mmWave radar
FRAME_SIZE = ADC_SAMPLES*LOOPS_PER_FRAME*NUM_TX*NUM_RX 
ANGLE_BINS = (ANGLE_RANGE * 2) // ANGLE_RES + 1

print("RANGE_RESOLUTION",RANGE_RESOLUTION)
print("ANGLE_BINS",ANGLE_BINS)
wavelength = 3e8 / (START_FREQ * 1e9) 
antenna_spacing = wavelength / 2  
num_elevation_bins = (ANGLE_RANGE * 2) // ANGLE_RES + 1
elevation_resolution = np.arcsin(1 / num_elevation_bins) * (180 / np.pi)  
print(f"Elevation Resolution: {elevation_resolution:.6f} degrees")

RANGE_RESOLUTION 0.04296015796840632
ANGLE_BINS 181
Elevation Resolution: 0.316553 degrees


In [3]:
# filename = "./datasets/radar_data/drone_2025-01-29_12_41_45_test.bin"
# # filename = "./datasetsFull/radar_data/drone_2025-01-29_13_10_41_test.bin"
filename = "./datasets/radar_data/adc_data_2025-04-08_05-49-07.npy"

In [5]:
adc_data = np.load(filename)
adc_data.shape

(100, 1118208)

In [5]:
NUM_FRAMES = 500
with open(filename, 'rb') as ADCBinFile: 
    frames = np.frombuffer(ADCBinFile.read( FRAME_SIZE*4*NUM_FRAMES), dtype=np.uint16)


# all_data = frame_reshape(frames, NUM_FRAMES)
adc_data = frames.reshape(NUM_FRAMES, -1)
print(adc_data.shape)


(500, 1118208)


In [6]:
all_data = np.apply_along_axis(DCA1000.organize, 1, adc_data, num_chirps= LOOPS_PER_FRAME* NUM_TX, num_rx= NUM_RX, num_samples= ADC_SAMPLES)
print(all_data.shape)



(100, 546, 4, 256)


In [7]:

def aoa_capon(x, steering_vector, magnitude=False):
    """Perform AOA estimation using Capon (MVDR) Beamforming on a rx by chirp slice

    Calculate the aoa spectrum via capon beamforming method using one full frame as input.
    This should be performed for each range bin to achieve AOA estimation for a full frame
    This function will calculate both the angle spectrum and corresponding Capon weights using
    the equations prescribed below.

    .. math::
        P_{ca} (\\theta) = \\frac{1}{a^{H}(\\theta) R_{xx}^{-1} a(\\theta)}
        
        w_{ca} (\\theta) = \\frac{R_{xx}^{-1} a(\\theta)}{a^{H}(\\theta) R_{xx}^{-1} a(\\theta)}

    Args:
        x (ndarray): Output of the 1d range fft with shape (num_ant, numChirps)
        steering_vector (ndarray): A 2D-array of size (numTheta, num_ant) generated from gen_steering_vec
        magnitude (bool): Azimuth theta bins should return complex data (False) or magnitude data (True). Default=False

    Raises:
        ValueError: steering_vector and or x are not the correct shape

    Returns:
        A list containing numVec and steeringVectors
        den (ndarray: A 1D-Array of size (numTheta) containing azimuth angle estimations for the given range
        weights (ndarray): A 1D-Array of size (num_ant) containing the Capon weights for the given input data
    
    Example:
        >>> # In this example, dataIn is the input data organized as numFrames by RDC
        >>> Frame = 0
        >>> dataIn = np.random.rand((num_frames, num_chirps, num_vrx, num_adc_samples))
        >>> for i in range(256):
        >>>     scan_aoa_capon[i,:], _ = dss.aoa_capon(dataIn[Frame,:,:,i].T, steering_vector, magnitude=True)

    """

    if steering_vector.shape[1] != x.shape[0]:
        raise ValueError("'steering_vector' with shape (%d,%d) cannot matrix multiply 'input_data' with shape (%d,%d)" \
        % (steering_vector.shape[0], steering_vector.shape[1], x.shape[0], x.shape[1]))
    # plt.imshow(np.abs(x), aspect='auto', cmap='viridis')
    # plt.savefig("./visualization/x.png", dpi=300, bbox_inches='tight')
    # plt.close()
    Rxx = cov_matrix(x)
    # plt.imshow(np.abs(Rxx), aspect='auto', cmap='viridis')
    # plt.savefig("./visualization/Rxx1.png", dpi=300, bbox_inches='tight')
    # plt.close()
    # print("After cov_matrix Radar_cube.shape",Rxx.shape)
    Rxx = forward_backward_avg(Rxx)
    # plt.imshow(np.abs(Rxx), aspect='auto', cmap='viridis')
    # plt.savefig("./visualization/Rxx2.png", dpi=300, bbox_inches='tight')
    # plt.close()
    # print("After forward_backward_avg Rxx.shape",Rxx.shape)

    Rxx_inv = np.linalg.inv(Rxx)
    # plt.imshow(np.abs(Rxx_inv), aspect='auto', cmap='viridis')
    # plt.savefig("./visualization/RxxInv.png", dpi=300, bbox_inches='tight')
    # plt.close()
    # print("After forward_backward_avg Rxx_inv.shape",Rxx.shape)

    # Calculate Covariance Matrix Rxx
    first = Rxx_inv @ steering_vector.T
    # print("first.shape",first.shape)
    # plt.imshow(np.abs(first), aspect='auto', cmap='viridis')
    # plt.savefig("./visualization/first.png", dpi=300, bbox_inches='tight')
    # plt.close()
    den = np.reciprocal(np.einsum('ij,ij->i', steering_vector.conj(), first.T))
    # plt.plot(np.abs(den))
    # plt.savefig("./visualization/den.png", dpi=300, bbox_inches='tight')
    # plt.close()
    # print("den.shape",den.shape)
    weights = np.matmul(first, den)
    # plt.plot(np.abs(weights))
    # plt.savefig("./visualization/weights.png", dpi=300, bbox_inches='tight')
    # plt.close()
    # print("weights.shape",weights.shape)
    if magnitude:
        return np.abs(den), weights
    else:
        return den, weights


# ------------------------------- HELPER FUNCTIONS -------------------------------

def cov_matrix(x):
    """ Calculates the spatial covariance matrix (Rxx) for a given set of input data (x=inputData). 
        Assumes rows denote Vrx axis.

    Args:
        x (ndarray): A 2D-Array with shape (rx, adc_samples) slice of the output of the 1D range fft

    Returns:
        Rxx (ndarray): A 2D-Array with shape (rx, rx)
    """
    
    if x.ndim > 2:
        raise ValueError("x has more than 2 dimensions.")

    if x.shape[0] > x.shape[1]:
        warnings.warn("cov_matrix input should have Vrx as rows. Needs to be transposed", RuntimeWarning)
        x = x.T

    _, num_adc_samples = x.shape
    Rxx = x @ np.conjugate(x.T)
    Rxx = np.divide(Rxx, num_adc_samples)

    return Rxx


def forward_backward_avg(Rxx):
    """ Performs forward backward averaging on the given input square matrix

    Args:
        Rxx (ndarray): A 2D-Array square matrix containing the covariance matrix for the given input data

    Returns:
        R_fb (ndarray): The 2D-Array square matrix containing the forward backward averaged covariance matrix
    """
    assert np.size(Rxx, 0) == np.size(Rxx, 1)

    # --> Calculation
    M = np.size(Rxx, 0)  # Find number of antenna elements
    Rxx = np.matrix(Rxx)  # Cast np.ndarray as a np.matrix

    # Create exchange matrix
    J = np.eye(M)  # Generates an identity matrix with row/col size M
    J = np.fliplr(J)  # Flips the identity matrix left right
    J = np.matrix(J)  # Cast np.ndarray as a np.matrix

    R_fb = 0.5 * (Rxx + J * np.conjugate(Rxx) * J)

    return np.array(R_fb)


def gen_steering_vec(ang_est_range, ang_est_resolution, num_ant,elevation=False,azimuth=False):
    """Generate a steering vector for AOA estimation given the theta range, theta resolution, and number of antennas

    Defines a method for generating steering vector data input --Python optimized Matrix format
    The generated steering vector will span from -angEstRange to angEstRange with increments of ang_est_resolution
    The generated steering vector should be used for all further AOA estimations (bartlett/capon)

    Args:
        ang_est_range (int): The desired span of thetas for the angle spectrum.
        ang_est_resolution (float): The desired resolution in terms of theta
        num_ant (int): The number of Vrx antenna signals captured in the RDC

    Returns:
        num_vec (int): Number of vectors generated (integer divide angEstRange/ang_est_resolution)
        steering_vectors (ndarray): The generated 2D-array steering vector of size (num_vec,num_ant)

    Example:
        >>> #This will generate a numpy array containing the steering vector with 
        >>> #angular span from -90 to 90 in increments of 1 degree for a 4 Vrx platform
        >>> _, steering_vec = gen_steering_vec(90,1,4)

    """
    elevationAntena = [2,3,4,5,8,9,10,11]
    azimuthAntena = [0,1,2,3,4,5,6,7]
    num_vec = (2 * ang_est_range / ang_est_resolution + 1)
    num_vec = int(round(num_vec))
    steering_vectors = np.zeros((num_vec, num_ant), dtype='complex64')
    if True:
        for kk in range(num_vec):
            for jj in range(num_ant):
                mag = -1 * np.pi * jj * np.sin((-ang_est_range + kk * ang_est_resolution) * np.pi / 180)
                real = np.cos(mag)
                imag = np.sin(mag)
                steering_vectors[kk, jj] = complex(real, imag)
    else:

        for kk in range(num_vec):

            for jj in range(num_ant):
                # print("kk:",kk,"jj:",jj,"azimuth:",azimuth,"elevation:",elevation)
                if jj in elevationAntena and elevation:
                    # print("Inside ele")
                    mag = -1 * np.pi * jj * np.sin((-ang_est_range + kk * ang_est_resolution) * np.pi / 180)
                    real = np.cos(mag)
                    imag = np.sin(mag)
                    steering_vectors[kk, jj] = complex(real, imag)
                if jj in azimuthAntena and azimuth:
                    # print("inside azi")
                    mag = -1 * np.pi * jj * np.sin((-ang_est_range + kk * ang_est_resolution) * np.pi / 180)
                    real = np.cos(mag)
                    imag = np.sin(mag)
                    steering_vectors[kk, jj] = complex(real, imag)

    return [num_vec, steering_vectors]

def compute_range_azimuth(radar_cube, angle_res=1, angle_range=90, method="apes"):

    n_range_bins = radar_cube.shape[2]
    n_rx = radar_cube.shape[1]
    n_chirps = radar_cube.shape[0]
    n_angle_bins = (angle_range * 2) // angle_res + 1

    range_cube = np.zeros_like(radar_cube)
    with objmode(range_cube="complex128[:,:,:]"):
        range_cube = np.fft.fft(radar_cube, axis=2)
    range_cube = np.transpose(range_cube, (2, 1, 0))
    range_cube = np.asarray(range_cube, dtype=np.complex64)

    range_cube_ = np.zeros(
        (range_cube.shape[0], range_cube.shape[1], range_cube.shape[2]),
        dtype=np.complex64,
    )

    _, steering_vec = gen_steering_vec(angle_range, angle_res, n_rx)
    # print(f"steering_vec shape: ", steering_vec.shape)
    range_azimuth = np.zeros((n_range_bins, n_angle_bins), dtype=np.complex_)
    beamWeights = np.zeros((n_range_bins,steering_vec.shape[1]), dtype=np.complex_)
    for r_idx in range(n_range_bins):
        range_cube_[r_idx] = range_cube[r_idx]
        steering_vec_ = steering_vec
        if method == "capon":
            # print("capon beamforming taking input: ", range_cube_[r_idx].shape, steering_vec_.shape, range_azimuth.shape)
            range_azimuth[r_idx, :], beamWeights[r_idx, :] = aoa_capon(range_cube_[r_idx], steering_vec_)
        else:
            raise ValueError("Unknown method")

    range_azimuth = np.log(np.abs(range_azimuth))
    # print("Weight.shape",_.shape)
    return range_azimuth,beamWeights

def _tdm(radar_cube, n_tx, n_rx):
    radar_cube_tdm = np.zeros(
        (radar_cube.shape[0] * n_tx, radar_cube.shape[1], radar_cube.shape[2]),
        dtype=np.complex64,
    )

    for i in range(n_tx):
        radar_cube_tdm[i::n_tx, i * n_rx : (i + 1) * n_rx] = radar_cube[
            :, i * n_rx : (i + 1) * n_rx
        ]

    return radar_cube_tdm



In [8]:
range_azimuth = np.zeros(( ANGLE_BINS,  ADC_SAMPLES))
range_elevation= np.zeros(( ANGLE_BINS,  ADC_SAMPLES))
num_vec, steering_vec = gen_steering_vec( ANGLE_RANGE,  ANGLE_RES,  8, azimuth=True)
num_vecElevation, steering_vecElevation = gen_steering_vec( ANGLE_RANGE,  ANGLE_RES, 4, elevation=True)
# num_vec, steering_vec = gen_steering_vec_2d( elevation_resolution,  ANGLE_RES,  VIRT_ANT)
tracker = EKF()
frameID = 0
pcd_datas = []
time_frames = []
rangeResult = []
dopplerResult = []
rangeAzimuthzResult = []
rawHeatmap = []
powerProfileValues = []
# start_time = filename.split('/')[-1].split('.')[0].split('drone_')[-1][:19]
# start_time_obj = datetime.strptime(start_time,'%Y-%m-%d_%H_%M_%S')

In [9]:
all_data.shape

(100, 546, 4, 256)

In [ ]:
frameID=0
heatmap_eleRes = []
heatmap_aziRes = []
for adc_data in all_data:
    # frameID+=1
    if frameID>0:
        print("frameID",frameID)

        # print("adc_data.shape",adc_data.shape)
        # radar_cube = dsp.range_processing(adc_data)
        radar_cube = adc_data
        # print("After range Processing radar_cube.shape", radar_cube.shape)
        mean = radar_cube.mean(0)
        # print("mean:",mean)
        radar_cube = radar_cube - mean 
        # print("After mean sub radar_cube.shape", radar_cube.shape)
        # rangeResult = radar_cube.sum(axis=(0,1))
        # plt.plot(np.abs(rangeResult))
        # plt.savefig("RangeRes.png")
        # plt.clf()
        # rangeResult.append(radar_cube)
        # ranges = np.arange(ADC_SAMPLES) * RANGE_RESOLUTION

        # --- capon beamforming
        beamWeights   = np.zeros(( VIRT_ANT,  ADC_SAMPLES), dtype=np.complex128)
        beamWeightsElevation   = np.zeros(( VIRT_ANT,  ADC_SAMPLES), dtype=np.complex128)
        # print("radar_cube.shape",radar_cube.shape)
        # print("steering_vec.shape",steering_vec.shape)
        

        radar_cube = np.concatenate((radar_cube[0::3,...], radar_cube[1::3,...], radar_cube[2::3,...]), axis=1)
        # print("After concat radar_cube.shape",radar_cube.shape) #(182, 12, 256)
        # radar_cube = radar_cube[:,:,20:100]
        radar_cube_a = radar_cube[:, :8, :]
        radar_cube_a = _tdm(radar_cube_a,2,4)
        # print("After tdm radar_cube_a.shape",radar_cube_a.shape) #(182, 12, 256)

        radar_cube_e = (radar_cube[:, 2:6, :] + radar_cube[:, 8:12, :]) / 2  # [2,3,4,5,8,9,10,11]
        radar_cube_e = _tdm(radar_cube_e,2,4)
        # print("After tdm radar_cube_e.shape",radar_cube_e.shape) #(182, 12, 256)
        # print("radar_cube_e.shape",radar_cube_e.shape)
        # print("radar_cube_a.shape",radar_cube_a.shape)

        heatmapAzi, beamWeights = compute_range_azimuth(radar_cube_a, 1, 90, "capon")
        heatmapEle, beamweightsEle = compute_range_azimuth(radar_cube_e, 1, 90, "capon")
        print("heatmapAzi.shape",heatmapAzi.shape)
        print("Beamweights.shape",beamWeights.shape)
        print("heatmapEle.shape",heatmapEle.shape)
        print("BeamweightsEle.shape",beamweightsEle.shape)
        # --- cfar in azimuth direction
        first_pass, _ = np.apply_along_axis(func1d=dsp.ca_,
                                            axis=0,
                                            arr=heatmapAzi,
                                            l_bound=.1,
                                            guard_len=3,
                                            noise_len=12)
        
        # --- cfar in range direction 
        second_pass, noise_floorFirst = np.apply_along_axis(func1d=dsp.ca_,
                                                    axis=0,
                                                    arr=heatmapAzi.T,
                                                    l_bound=.2,
                                                    guard_len=3,
                                                    noise_len=12)
        # --- cfar in azimuth direction
        third_pass, _ = np.apply_along_axis(func1d=dsp.ca_,
                                            axis=0,
                                            arr=heatmapEle,
                                            l_bound=.1,
                                            guard_len=3,
                                            noise_len=12)
        
        # --- cfar in range direction
        fourth_pass, noise_floorSecond = np.apply_along_axis(func1d=dsp.ca_,
                                                    axis=0,
                                                    arr=heatmapEle.T,
                                                    l_bound=.2,
                                                    guard_len=3,
                                                    noise_len=12)
        
        # print("first_pass:",first_pass.shape,"second_pass:",second_pass.shape,"third_pass:",third_pass.shape,)
            # --- classify peaks and caclulate snrs
        

        # pairsAziNoCFAR = np.argwhere(heatmapAzi > 0)  
        # azimuthsNoCFAR , rangesAziNoCFAR  = pairsAziNoCFAR .T  
        # snrsAziNoCFAR  = heatmapAzi[pairsAziNoCFAR [:, 0], pairsAziNoCFAR [:, 1]] - noise_floorFirst[pairsAziNoCFAR [:, 0], pairsAziNoCFAR [:, 1]]
        # pairsEleNoCFAR  = np.argwhere(heatmapEle > 0)  
        # elevationsNoCFAR , rangesEleNoCFAR  = pairsEle.T  
        # snrsEleNoCFAR  = heatmapEle[pairsEleNoCFAR [:, 0], pairsEleNoCFAR [:, 1]] - noise_floorSecond[pairsEleNoCFAR [:, 0], pairsEleNoCFAR [:, 1]]
        
        noise_floorFirst = noise_floorFirst.T
        noise_floorSecond = noise_floorSecond.T

        ############CFAR############
        first_pass = (heatmapAzi > first_pass)
        second_pass = (heatmapAzi > second_pass.T)
        third_pass = (heatmapEle > third_pass)
        fourth_pass = (heatmapEle > fourth_pass.T)
 
        peaksAzi = (first_pass & second_pass)
        # peaksAzi[: SKIP_SIZE, :] = 0
        # peaksAzi[- SKIP_SIZE:, :] = 0
        # peaksAzi[:, : SKIP_SIZE] = 0
        # peaksAzi[:, -SKIP_SIZE:] = 0
        pairsAzi = np.argwhere(peaksAzi)

        peaksEle = (third_pass & fourth_pass)
        # peaksEle[: SKIP_SIZE, :] = 0
        # peaksEle[- SKIP_SIZE:, :] = 0
        # peaksEle[:, : SKIP_SIZE] = 0
        # peaksEle[:, -SKIP_SIZE:] = 0
        pairsEle = np.argwhere(peaksEle)

        

        # pairsAzi = np.argwhere(heatmapAzi > 0)
        print("pairsAzi",pairsAzi.shape)# NO cfaR (46336, 2)
        azimuths, rangesAzi = pairsAzi.T
        print("noise_floorFirst:",noise_floorFirst.shape)

        powerProfile = np.sum(np.abs(radar_cube) ** 2, axis=(0, 1))  
        powerProfile  = powerProfile[rangesAzi]
        print("powerProfile:",len(powerProfile))
        snrsAzi = heatmapAzi[pairsAzi[:,0], pairsAzi[:,1]] - noise_floorFirst[pairsAzi[:,0], pairsAzi[:,1]]

        # pairsEle = np.argwhere(heatmapEle > 0)
        elevations, rangesEle = pairsEle.T
        snrsEle = heatmapEle[pairsEle[:,0], pairsEle[:,1]] - noise_floorSecond[pairsEle[:,0], pairsEle[:,1]]

         # --- convert bins to units
        ranges = rangesAzi *  RANGE_RESOLUTION
        azimuths = (azimuths - ( NUM_ANGLE_BINS // 2)) * (np.pi / 180)
        elevations = (elevations - ( NUM_ANGLE_BINS // 2)) * (np.pi / 180)
        snrs = snrsAzi
        print("rangesAzi:",len(rangesAzi),"rangesEle:",len(rangesEle),"ranges:",len(ranges),"azimuths:", len(azimuths),"elevations:",len(elevations))

        matching_values = np.intersect1d(rangesAzi, rangesEle)
        matching_indices = [np.where(rangesAzi == value)[0][0] for value in matching_values]
        matching_ranges = rangesAzi[matching_indices]
        matching_azimuth = azimuths[matching_indices]
        matching_elevations = elevations[matching_indices]
        matching_powers = powerProfile[matching_indices]
        print("Matching Indices:", len(matching_indices))
        print("Matching Elevations:", matching_elevations)
        print("matching azimuths: ", matching_azimuth)
        matching_ranges = matching_ranges*RANGE_RESOLUTION 
        x = matching_ranges * np.cos(matching_elevations) * np.sin(matching_azimuth)
        y = matching_ranges * np.cos(matching_elevations) * np.cos(matching_azimuth)
        z = - matching_ranges * np.sin(matching_elevations)

        points = np.column_stack((x, y, z))
        fig = plt.figure(figsize=(10, 7))
        ax = fig.add_subplot(111, projection='3d')
        scatter = ax.scatter(x, y, z, c=matching_powers, cmap='viridis', marker='o')
        r = np.linspace(-10, 10, 20)
        X, Y = np.meshgrid(r, r)

        # Plane z = 0
        Z = np.zeros_like(X)
        ax.plot_surface(X, Y, Z, color='red', alpha=0.3, label='z=0')

        # Plane x = 0
        Z = np.linspace(-10, 10, 20)
        Y, Z = np.meshgrid(r, r)
        X = np.zeros_like(Y)
        ax.plot_surface(X, Y, Z, color='green', alpha=0.3, label='x=0')

        # Plane y = 0
        X, Z = np.meshgrid(r, r)
        Y = np.zeros_like(X)
        ax.plot_surface(X, Y, Z, color='blue', alpha=0.3, label='y=0')

        ax.set_xlabel('X')
        ax.set_ylabel('Y')
        ax.set_zlabel('Z')
        ax.set_xlim([-10, 10])  
        ax.set_ylim([0, 12])  
        ax.set_zlim([-10, 10]) 
        ax.set_title('Point Cloud Visualization')
        plt.savefig(f"./visualization/pcd/{frameID}.png")
        plt.clf()


        #  --- put into EKF
        # tracker.update_point_cloud(ranges, azimuths, elevations)
        # targetDescr, tNum = tracker.step()
        # frame_pcd = np.zeros((len(tracker.point_cloud),6))
        # frame_pcd = np.zeros((len(tracker.point_cloud),3))
        # for point_cloud, idx in zip(tracker.point_cloud, range(len(tracker.point_cloud))):
        #     frame_pcd[idx,0] = -np.sin(point_cloud.angle) * point_cloud.range
        #     frame_pcd[idx,1] = np.cos(point_cloud.angle) * point_cloud.range
        #     frame_pcd[idx,2] = np.cos(point_cloud.angle) * point_cloud.range
        #     frame_pcd[idx,3] = point_cloud.doppler 
        #     frame_pcd[idx,4] = point_cloud.snr
        #     frame_pcd[idx,5] = point_cloud.range
        #     frame_pcd[idx,6] = point_cloud.angle
        #     frame_pcd[idx,7] = point_cloud.power

        # break
    # if frameID==149:
    #     break
    frameID+=1
    
        
   

frameID 1
heatmapAzi.shape (256, 181)
Beamweights.shape (256, 8)
heatmapEle.shape (256, 181)
BeamweightsEle.shape (256, 4)
pairsAzi (1390, 2)
noise_floorFirst: (256, 181)
powerProfile: 1390
rangesAzi: 1390 rangesEle: 2271 ranges: 1390 azimuths: 1390 elevations: 2271
Matching Indices: 134
Matching Elevations: [ 0.85521133  0.85521133  0.78539816  0.80285146  0.80285146  0.80285146
  0.80285146  0.45378561  0.45378561  0.45378561  0.33161256  0.33161256
  0.33161256  0.33161256  0.33161256  0.33161256  0.34906585  0.34906585
  0.34906585  0.34906585  0.34906585  0.34906585  0.34906585  0.34906585
  0.34906585  0.50614548  0.45378561  0.48869219  0.48869219  0.2268928
 -0.50614548 -0.50614548 -0.50614548 -0.50614548 -0.50614548 -0.50614548
 -0.50614548 -0.50614548 -0.50614548 -0.50614548 -0.50614548 -0.50614548
 -0.50614548 -0.48869219 -0.48869219 -0.48869219 -0.48869219 -0.48869219
 -0.48869219 -0.38397244 -0.36651914 -0.36651914  0.05235988  0.05235988
  0.38397244  0.38397244  0.383972

/tmp/ipykernel_1156724/1357258375.py:153: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig = plt.figure(figsize=(10, 7))


frameID 22
heatmapAzi.shape (256, 181)
Beamweights.shape (256, 8)
heatmapEle.shape (256, 181)
BeamweightsEle.shape (256, 4)
pairsAzi (1128, 2)
noise_floorFirst: (256, 181)
powerProfile: 1128
rangesAzi: 1128 rangesEle: 2130 ranges: 1128 azimuths: 1128 elevations: 2130
Matching Indices: 111
Matching Elevations: [ 0.57595865  0.57595865  0.57595865  0.57595865  0.57595865 -0.50614548
 -0.50614548 -0.50614548 -0.50614548 -0.50614548 -0.50614548 -0.50614548
 -0.50614548 -0.50614548 -0.50614548 -0.50614548 -0.50614548 -0.50614548
 -0.48869219 -0.55850536 -0.55850536 -0.55850536 -0.55850536 -0.54105207
 -0.54105207 -0.31415927 -0.31415927  0.34906585  0.33161256  0.33161256
  0.33161256  0.33161256  0.03490659  0.05235988  0.26179939 -0.05235988
 -0.05235988 -0.05235988 -0.05235988 -0.05235988 -0.05235988 -0.05235988
 -0.05235988 -0.05235988 -0.05235988 -0.05235988  0.06981317 -0.54105207
 -0.54105207 -0.54105207 -0.54105207 -0.54105207 -0.54105207 -0.54105207
 -0.54105207 -0.52359878 -0.5235

<Figure size 1000x700 with 0 Axes>

<Figure size 1000x700 with 0 Axes>

...

<Figure size 1000x700 with 0 Axes>

<Figure size 1000x700 with 0 Axes>

<Figure size 1000x700 with 0 Axes>

<Figure size 1000x700 with 0 Axes>

<Figure size 1000x700 with 0 Axes>

In [ ]:
 # Note that when replacing with generic doppler estimation functions, radarCube is interleaved and
    # has doppler at the last dimension.
    # for i in range( ADC_SAMPLES):
        # range_azimuth[:,i], beamWeights[:,i] = aoa_capon(radar_cube[:, :, i].T, steering_vec, magnitude=True)
    #     break
    # break

    #did capon oonly on specific antena 9.10.11.12

    # for i in range( ADC_SAMPLES):
    #     range_elevation[:,i], beamWeightsElevation[:,i] = dsp.aoa_capon(radar_cube[:, :, i].T, steering_vecElevation, magnitude=True)
    # print("range_azimuth.shape",range_azimuth.shape)
    # print("range_elevation.shape",range_elevation.shape)
    # print("beamWeights.shape",beamWeights.shape)
    # """ 3 (Object Detection) """
    # heatmap_log = np.log2(range_azimuth)
 
    # #make it log scale
    # heatmap_logElevation = np.log2(range_elevation)
    # # azimuthElevation = np.concatenate((heatmap_log,heatmap_logElevation),axis=0)
    # # azimuthElevation = np.zeros((256, 256))
    # azimuthElevation = np.matmul(heatmap_log,heatmap_logElevation.T)
    # # for i in range(range_azimuth.shape[0]):
    # #     azimuthElevation += np.outer(range_azimuth[i], range_elevation[i])
    # print("azimuthElevation.shape",azimuthElevation.shape)
    # plt.imshow(azimuthElevation, aspect='auto', cmap='viridis')
    # plt.savefig("./visualization/azimuthElevation.png", dpi=300, bbox_inches='tight')
    # plt.close()
    
    # fig, axs = plt.subplots(1, 2, figsize=(12, 6))
    # axs[0].imshow(heatmap_log, aspect='auto', cmap='viridis')
    # axs[0].set_title('Azimuth Heatmap')
    # axs[1].imshow(heatmap_logElevation, aspect='auto', cmap='viridis')
    # axs[1].set_title('Elevation Heatmap')
    # plt.savefig("./visualization/combined_heatmaps.png", dpi=300, bbox_inches='tight')
    # plt.close()
    # # plt.show()
    # # --- cfar in azimuth direction
    # first_pass, _ = np.apply_along_axis(func1d=dsp.ca_,
    #                                     axis=0,
    #                                     arr=heatmap_log,
    #                                     l_bound=1.5,
    #                                     guard_len=4,
    #                                     noise_len=16)
    
    # # --- cfar in range direction
    # second_pass, noise_floor = np.apply_along_axis(func1d=dsp.ca_,
    #                                             axis=0,
    #                                             arr=heatmap_log.T,
    #                                             l_bound=2.5,
    #                                             guard_len=4,
    #                                             noise_len=16)
    # # --- cfar in elevation direction
    # first_passElevation, _Elevation = np.apply_along_axis(func1d=dsp.ca_,
    #                                     axis=0,
    #                                     arr=heatmap_logElevation,
    #                                     l_bound=1.5,
    #                                     guard_len=4,
    #                                     noise_len=16)
    
    # # --- cfar in range direction
    # second_passElevation, noise_floorElevation = np.apply_along_axis(func1d=dsp.ca_,
    #                                             axis=0,
    #                                             arr=heatmap_logElevation.T,
    #                                             l_bound=2.5,
    #                                             guard_len=4,
    #                                             noise_len=16)



    # # --- classify peaks and caclulate snrs
    # noise_floor = noise_floor.T
    # first_pass = (heatmap_log > first_pass)
    # second_pass = (heatmap_log > second_pass.T)
    # peaks = (first_pass & second_pass)
    # peaks[: SKIP_SIZE, :] = 0
    # peaks[- SKIP_SIZE:, :] = 0
    # peaks[:, : SKIP_SIZE] = 0
    # peaks[:, -SKIP_SIZE:] = 0
    # pairs = np.argwhere(peaks)
    # azimuths, ranges = pairs.T
    # snrs = heatmap_log[pairs[:,0], pairs[:,1]] - noise_floor[pairs[:,0], pairs[:,1]]


    # noise_floorElevation = noise_floorElevation.T
    # first_passElevation = (heatmap_logElevation > first_passElevation)
    # second_passElevation = (heatmap_logElevation > second_passElevation.T)
    # peaksElevation = (first_passElevation & second_passElevation)
    # peaksElevation[: SKIP_SIZE, :] = 0
    # peaksElevation[- SKIP_SIZE:, :] = 0
    # peaksElevation[:, : SKIP_SIZE] = 0
    # peaksElevation[:, -SKIP_SIZE:] = 0
    # pairsElevation = np.argwhere(peaksElevation)
    # elevations, ranges = pairsElevation.T
    # snrsElevation = heatmap_logElevation[pairsElevation[:,0], pairsElevation[:,1]] - noise_floorElevation[pairsElevation[:,0], pairsElevation[:,1]]

    # """ 4 (Doppler Estimation) """

    # # --- get peak indices
    # # beamWeights should be selected based on the range indices from CFAR.
    # dopplerFFTInput = radar_cube[:, :, ranges]
    # beamWeights  = beamWeights[:, ranges]

    # elevationFFTInput = radar_cube[:, :, ranges]
    # beamWeightsElevation = beamWeightsElevation[:, ranges]

    # powerProfile = np.sum(np.abs(radar_cube) ** 2, axis=(0, 1))  
    # powerProfile  = powerProfile[ranges]

    # # --- estimate doppler values
    # # For each detected object and for each chirp combine the signals from 4 Rx, i.e.
    # # For each detected object, matmul (numChirpsPerFrame, numRxAnt) with (numRxAnt) to (numChirpsPerFrame)
    # print("dopplerFFTInput:",dopplerFFTInput.shape,"elevationFFTInput:",elevationFFTInput.shape)
    # dopplerFFTInput = np.einsum('ijk,jk->ik', dopplerFFTInput, beamWeights)
    # if not dopplerFFTInput.shape[-1]:
    #     continue


    # dopplerEst = np.fft.fft(dopplerFFTInput, axis=0)
    # dopplerEst = np.argmax(dopplerEst, axis=0)
    # dopplerEst[dopplerEst[:]>= LOOPS_PER_FRAME/2] -=  LOOPS_PER_FRAME
    

    # """ 5 (Extended Kalman Filter) """

    # # --- convert bins to units
    # ranges = ranges *  RANGE_RESOLUTION
    # azimuths = (azimuths - ( NUM_ANGLE_BINS // 2)) * (np.pi / 180)
    # elevations = (elevations - ( NUM_ANGLE_BINS // 2)) * (np.pi / 180)
    # dopplers = dopplerEst *  DOPPLER_RESOLUTION
    # snrs = snrs
    # # rangeAzimuthzResult.append([ranges,azimuths])
    # print(f"ranges: {ranges.shape}, azimuths: {azimuths.shape},elevation: {elevations.shape}, dopplers: {dopplers.shape}, snrs: {snrs.shape}, powerProfile: {powerProfile.shape}")
    
    # concatenated= np.concatenate((heatmap_log, heatmap_logElevation), axis=1)
    # x = np.arange(concatenated.shape[1])  # X-coordinates
    # y = np.arange(concatenated.shape[0])  # Y-coordinates
    # X, Y = np.meshgrid(x, y)
    # Z = concatenated
    # fig = plt.figure(figsize=(12, 8))
    # ax = fig.add_subplot(111, projection='3d')
    # surf = ax.plot_surface(X, Y, Z, cmap='viridis')
    # ax.set_xlabel('X-axis')
    # ax.set_ylabel('Y-axis')
    # ax.set_zlabel('Z-axis')
    # fig.colorbar(surf, ax=ax, shrink=0.5, aspect=10)

    # break
    # print(powerProfile)
    # powerProfileValues.append(powerProfile)

    # --- put into EKF
    # tracker.update_point_cloud(ranges, azimuths, elevations,dopplers, snrs, powerProfile)
    # targetDescr, tNum = tracker.step()
    # # frame_pcd = np.zeros((len(tracker.point_cloud),6))
    # frame_pcd = np.zeros((len(tracker.point_cloud),8))
    # for point_cloud, idx in zip(tracker.point_cloud, range(len(tracker.point_cloud))):
    #     frame_pcd[idx,0] = -np.sin(point_cloud.angle) * point_cloud.range
    #     frame_pcd[idx,1] = np.cos(point_cloud.angle) * point_cloud.range
    #     frame_pcd[idx,2] = np.cos(point_cloud.angle) * point_cloud.range
    #     frame_pcd[idx,3] = point_cloud.doppler 
    #     frame_pcd[idx,4] = point_cloud.snr
    #     frame_pcd[idx,5] = point_cloud.range
    #     frame_pcd[idx,6] = point_cloud.angle
    #     frame_pcd[idx,7] = point_cloud.power

        
#     pcd_datas.append(frame_pcd)
#     rawHeatmap.append(heatmap_log)
# pointcloud = np.array(pcd_datas)


In [ ]:
frameID=0
heatmap_eleRes = []
heatmap_aziRes = []
for adc_data in all_data:
    # time_current = start_time_obj+timedelta(seconds=frameID*(200)/1000)
    # time_frames.append(time_current.strftime('%Y-%m-%d %H_%M_%S.%f'))
    # frameID+=1
    if frameID>100:
        print("frameID",frameID)
        # print("adc_data.shape",adc_data.shape)
        # radar_cube = dsp.range_processing(adc_data)
        radar_cube = adc_data
        # print("After range Processing radar_cube.shape", radar_cube.shape)
        mean = radar_cube.mean(0)
        # print("mean:",mean)
        radar_cube = radar_cube - mean 
        print("After mean sub radar_cube.shape", radar_cube.shape)
        rangeResult = radar_cube.sum(axis=(0,1))
        plt.plot(np.abs(rangeResult))
        plt.savefig("RangeRes.png")
        plt.clf()
        # rangeResult.append(radar_cube)
        ranges = np.arange(ADC_SAMPLES) * RANGE_RESOLUTION

        # plt.plot(ranges,np.abs(rangeResult))
        # plt.xlabel('Range (meters)')
        # plt.ylabel('Reflected Power')
        # plt.title('Interpreting a Single Chirp')
        # plt.show()
        # separate_tx_radar_cube = separate_tx(radar_cube,3,1,0)
        # doppler_fft_value = np.fft.fft(separate_tx_radar_cube, axis=0)
        # doppler_fft_value = np.fft.fftshift(doppler_fft_value, axes=0)
        # doppler_fft_value = np.abs(doppler_fft_value.sum(axis=1))
        # doppler_fft_value_scaled = (doppler_fft_value-np.min(doppler_fft_value))/(np.max(doppler_fft_value)-np.min(doppler_fft_value))
        # dopplerResult.append(doppler_fft_value_scaled)


        # --- capon beamforming
        beamWeights   = np.zeros(( VIRT_ANT,  ADC_SAMPLES), dtype=np.complex128)
        beamWeightsElevation   = np.zeros(( VIRT_ANT,  ADC_SAMPLES), dtype=np.complex128)
        # print("radar_cube.shape",radar_cube.shape)
        # print("steering_vec.shape",steering_vec.shape)
        

        radar_cube = np.concatenate((radar_cube[0::3,...], radar_cube[1::3,...], radar_cube[2::3,...]), axis=1)
        # print("After concat radar_cube.shape",radar_cube.shape) #(182, 12, 256)
        # radar_cube = radar_cube[:,:,20:100]
        radar_cube_a = radar_cube[:, :8, :]
        radar_cube_a = _tdm(radar_cube_a,2,4)
        # print("After tdm radar_cube_a.shape",radar_cube_a.shape) #(182, 12, 256)

        radar_cube_e = (radar_cube[:, 2:6, :] + radar_cube[:, 8:12, :]) / 2  # [2,3,4,5,8,9,10,11]
        radar_cube_e = _tdm(radar_cube_e,2,4)
        # print("After tdm radar_cube_e.shape",radar_cube_e.shape) #(182, 12, 256)
        # print("radar_cube_e.shape",radar_cube_e.shape)
        # print("radar_cube_a.shape",radar_cube_a.shape)


        heatmap_ele = np.squeeze(np.stack(
            [
                compute_range_azimuth(
                    radar_cube_e, 1, 90, "capon"
                ),
            ]
        ), axis=0)
        heatmap_azim = np.squeeze(np.stack(
            [
                compute_range_azimuth(
                    radar_cube_a, 1, 90, "capon"
                ),
            ]
        ), axis=0)
        plt.imshow(np.abs(heatmap_azim))
        plt.savefig("heatmapAzim.png")
        plt.clf()
        plt.imshow(np.abs(heatmap_ele))
        plt.savefig("heatmapEle.png")
        plt.clf()
        range_bins = 256
        azimuth_bins = 181 
        elevation_bins = 181
        volume = np.zeros((range_bins, azimuth_bins, elevation_bins))
        for r in range(range_bins):
            volume[r] = np.outer(heatmap_azim[r], heatmap_ele[r])

        # Choose a range index to visualize a 2D surface in azimuth-elevation at that range
        r_idx = 50  # any value in 0 to range_bins-1
        Z = volume[r_idx]  # shape (azimuth_bins, elevation_bins)

        # Create meshgrid for azimuth and elevation
        azimuth = np.linspace(-90, 90, azimuth_bins)
        elevation = np.linspace(-90, 90, elevation_bins)
        A, E = np.meshgrid(azimuth, elevation)

        fig = plt.figure(figsize=(10, 8))
        ax = fig.add_subplot(111, projection='3d')

        # Plot surface
        surf = ax.plot_surface(A, E, Z.T, cmap='viridis', edgecolor='none')
        ax.set_title(f"Range-Azimuth-Elevation Surface at Range Bin {r_idx}")
        ax.set_xlabel("Azimuth (deg)")
        ax.set_ylabel("Elevation (deg)")
        ax.set_zlabel("Intensity")
        fig.colorbar(surf, ax=ax, shrink=0.5, aspect=10, label="Power")

        plt.tight_layout()
        plt.savefig(f"./visualization/merged/{frameID}.png")
        plt.clf()
        # heatmap_azim = np.stack([heatmap_azim]*181,axis=2)
        # heatmap_ele = np.stack([heatmap_ele]*181,axis=2)

        # heatmapAimEle = heatmap_azim*heatmap_ele

        # print(heatmapAimEle.shape)
        # fig = plt.figure(figsize=(10, 8))
        # ax = fig.add_subplot(111, projection='3d')
        # ax.voxels(heatmapAimEle, facecolors='blue', edgecolor='k')

        # ax.set_xlabel('X')
        # ax.set_ylabel('Y')
        # ax.set_zlabel('Z')
        # num_range_bins = 256
        # num_angle_bins = 181
        # theta_deg = np.linspace(-90, 90, num_angle_bins)
        # phi_deg = np.linspace(-90, 90, num_angle_bins)
        # theta = np.radians(theta_deg)
        # phi = np.radians(phi_deg)

        # # Step 2: Create meshgrids
        # r = np.arange(num_range_bins)  # or actual range values in meters, if available

        # # r-theta → azimuth
        # r_grid_theta, theta_grid = np.meshgrid(r, theta, indexing="ij")  # shape (256, 181)

        # # r-phi → elevation
        # r_grid_phi, phi_grid = np.meshgrid(r, phi, indexing="ij")  # shape (256, 181)

        # # Optional: Average both heatmaps to get a single magnitude matrix
        # magnitude = (range_azimuth + range_elevation) / 2  # shape (256, 181)

        # # Step 3: Compute x, y, z
        # x = r_grid_phi * np.cos(phi_grid) * np.sin(theta_grid)
        # y = r_grid_phi * np.cos(phi_grid) * np.cos(theta_grid)
        # z = r_grid_phi * np.sin(phi_grid)
        # fig = plt.figure()
        # ax = fig.add_subplot(111, projection='3d')

        # # Flatten and plot as point cloud
        # ax.scatter(x.flatten(), y.flatten(), z.flatten(), c=magnitude.flatten(), cmap='viridis', s=1)

        # ax.set_xlabel('X')
        # ax.set_ylabel('Y')
        # ax.set_zlabel('Z')

        # # Parameters
        # max_range_meters = 10  # change based on your radar config
        # ranges = np.linspace(0, max_range_meters, 256)
        # angles_deg = np.linspace(-90, 90, 181)
        # angles_rad = np.deg2rad(angles_deg)

        # # Create meshgrid of range and angle
        # R, A = np.meshgrid(ranges, angles_rad, indexing='ij')  # shape (256, 181)

        # # Convert Azimuth heatmap to Cartesian X, Y (2D)
        # X = R * np.cos(A)
        # Y = R * np.sin(A)

        # # Convert Elevation heatmap to Z (we assume same angles as azimuth for simplicity)
        # Z = heatmap_ele  # You can also scale this or apply log-scaling

        # # Optional: combine the two heatmaps (e.g., by multiplying or averaging)
        # combined_intensity = (heatmap_azim + heatmap_ele) / 2

        # # Plot 3D surface
        # fig = plt.figure(figsize=(12, 8))
        # ax = fig.add_subplot(111, projection='3d')
        # surf = ax.plot_surface(X, Y, Z, facecolors=plt.cm.viridis(combined_intensity / combined_intensity.max()), rstride=1, cstride=1, linewidth=0, antialiased=False, shade=False)

        # ax.set_xlabel('X (meters)')
        # ax.set_ylabel('Y (meters)')
        # ax.set_zlabel('Z (elevation)')
        # ax.set_title('3D Range-Azimuth-Elevation Heatmap')
        # plt.tight_layout()
        # plt.savefig(f"./visualization/merged/{frameID}.png")
        # plt.clf()


        heatmap_aziRes.append(heatmap_azim)
        heatmap_eleRes.append(heatmap_ele)
        # print("heatmap_ele.shape",heatmap_ele.shape)
        # print("heatmap_azim.shape",heatmap_azim.shape)

    if frameID==150:
        break
    frameID+=1
    # Note that when replacing with generic doppler estimation functions, radarCube is interleaved and
    # has doppler at the last dimension.
    # for i in range( ADC_SAMPLES):
    #     range_azimuth[:,i], beamWeights[:,i] = aoa_capon(radar_cube[:, :, i].T, steering_vec, magnitude=True)
    #     break
    # break

    #did capon oonly on specific antena 9.10.11.12

    # for i in range( ADC_SAMPLES):
    #     range_elevation[:,i], beamWeightsElevation[:,i] = dsp.aoa_capon(radar_cube[:, :, i].T, steering_vecElevation, magnitude=True)
    # print("range_azimuth.shape",range_azimuth.shape)
    # print("range_elevation.shape",range_elevation.shape)
    # print("beamWeights.shape",beamWeights.shape)
    # """ 3 (Object Detection) """
    # heatmap_log = np.log2(range_azimuth)
 
    # #make it log scale
    # heatmap_logElevation = np.log2(range_elevation)
    # # azimuthElevation = np.concatenate((heatmap_log,heatmap_logElevation),axis=0)
    # # azimuthElevation = np.zeros((256, 256))
    # azimuthElevation = np.matmul(heatmap_log,heatmap_logElevation.T)
    # # for i in range(range_azimuth.shape[0]):
    # #     azimuthElevation += np.outer(range_azimuth[i], range_elevation[i])
    # print("azimuthElevation.shape",azimuthElevation.shape)
    # plt.imshow(azimuthElevation, aspect='auto', cmap='viridis')
    # plt.savefig("./visualization/azimuthElevation.png", dpi=300, bbox_inches='tight')
    # plt.close()
    
    # fig, axs = plt.subplots(1, 2, figsize=(12, 6))
    # axs[0].imshow(heatmap_log, aspect='auto', cmap='viridis')
    # axs[0].set_title('Azimuth Heatmap')
    # axs[1].imshow(heatmap_logElevation, aspect='auto', cmap='viridis')
    # axs[1].set_title('Elevation Heatmap')
    # plt.savefig("./visualization/combined_heatmaps.png", dpi=300, bbox_inches='tight')
    # plt.close()
    # # plt.show()
    # # --- cfar in azimuth direction
    # first_pass, _ = np.apply_along_axis(func1d=dsp.ca_,
    #                                     axis=0,
    #                                     arr=heatmap_log,
    #                                     l_bound=1.5,
    #                                     guard_len=4,
    #                                     noise_len=16)
    
    # # --- cfar in range direction
    # second_pass, noise_floor = np.apply_along_axis(func1d=dsp.ca_,
    #                                             axis=0,
    #                                             arr=heatmap_log.T,
    #                                             l_bound=2.5,
    #                                             guard_len=4,
    #                                             noise_len=16)
    # # --- cfar in elevation direction
    # first_passElevation, _Elevation = np.apply_along_axis(func1d=dsp.ca_,
    #                                     axis=0,
    #                                     arr=heatmap_logElevation,
    #                                     l_bound=1.5,
    #                                     guard_len=4,
    #                                     noise_len=16)
    
    # # --- cfar in range direction
    # second_passElevation, noise_floorElevation = np.apply_along_axis(func1d=dsp.ca_,
    #                                             axis=0,
    #                                             arr=heatmap_logElevation.T,
    #                                             l_bound=2.5,
    #                                             guard_len=4,
    #                                             noise_len=16)



    # # --- classify peaks and caclulate snrs
    # noise_floor = noise_floor.T
    # first_pass = (heatmap_log > first_pass)
    # second_pass = (heatmap_log > second_pass.T)
    # peaks = (first_pass & second_pass)
    # peaks[: SKIP_SIZE, :] = 0
    # peaks[- SKIP_SIZE:, :] = 0
    # peaks[:, : SKIP_SIZE] = 0
    # peaks[:, -SKIP_SIZE:] = 0
    # pairs = np.argwhere(peaks)
    # azimuths, ranges = pairs.T
    # snrs = heatmap_log[pairs[:,0], pairs[:,1]] - noise_floor[pairs[:,0], pairs[:,1]]


    # noise_floorElevation = noise_floorElevation.T
    # first_passElevation = (heatmap_logElevation > first_passElevation)
    # second_passElevation = (heatmap_logElevation > second_passElevation.T)
    # peaksElevation = (first_passElevation & second_passElevation)
    # peaksElevation[: SKIP_SIZE, :] = 0
    # peaksElevation[- SKIP_SIZE:, :] = 0
    # peaksElevation[:, : SKIP_SIZE] = 0
    # peaksElevation[:, -SKIP_SIZE:] = 0
    # pairsElevation = np.argwhere(peaksElevation)
    # elevations, ranges = pairsElevation.T
    # snrsElevation = heatmap_logElevation[pairsElevation[:,0], pairsElevation[:,1]] - noise_floorElevation[pairsElevation[:,0], pairsElevation[:,1]]

    # """ 4 (Doppler Estimation) """

    # # --- get peak indices
    # # beamWeights should be selected based on the range indices from CFAR.
    # dopplerFFTInput = radar_cube[:, :, ranges]
    # beamWeights  = beamWeights[:, ranges]

    # elevationFFTInput = radar_cube[:, :, ranges]
    # beamWeightsElevation = beamWeightsElevation[:, ranges]

    # powerProfile = np.sum(np.abs(radar_cube) ** 2, axis=(0, 1))  
    # powerProfile  = powerProfile[ranges]

    # # --- estimate doppler values
    # # For each detected object and for each chirp combine the signals from 4 Rx, i.e.
    # # For each detected object, matmul (numChirpsPerFrame, numRxAnt) with (numRxAnt) to (numChirpsPerFrame)
    # print("dopplerFFTInput:",dopplerFFTInput.shape,"elevationFFTInput:",elevationFFTInput.shape)
    # dopplerFFTInput = np.einsum('ijk,jk->ik', dopplerFFTInput, beamWeights)
    # if not dopplerFFTInput.shape[-1]:
    #     continue


    # dopplerEst = np.fft.fft(dopplerFFTInput, axis=0)
    # dopplerEst = np.argmax(dopplerEst, axis=0)
    # dopplerEst[dopplerEst[:]>= LOOPS_PER_FRAME/2] -=  LOOPS_PER_FRAME
    

    # """ 5 (Extended Kalman Filter) """

    # # --- convert bins to units
    # ranges = ranges *  RANGE_RESOLUTION
    # azimuths = (azimuths - ( NUM_ANGLE_BINS // 2)) * (np.pi / 180)
    # elevations = (elevations - ( NUM_ANGLE_BINS // 2)) * (np.pi / 180)
    # dopplers = dopplerEst *  DOPPLER_RESOLUTION
    # snrs = snrs
    # # rangeAzimuthzResult.append([ranges,azimuths])
    # print(f"ranges: {ranges.shape}, azimuths: {azimuths.shape},elevation: {elevations.shape}, dopplers: {dopplers.shape}, snrs: {snrs.shape}, powerProfile: {powerProfile.shape}")
    
    # concatenated= np.concatenate((heatmap_log, heatmap_logElevation), axis=1)
    # x = np.arange(concatenated.shape[1])  # X-coordinates
    # y = np.arange(concatenated.shape[0])  # Y-coordinates
    # X, Y = np.meshgrid(x, y)
    # Z = concatenated
    # fig = plt.figure(figsize=(12, 8))
    # ax = fig.add_subplot(111, projection='3d')
    # surf = ax.plot_surface(X, Y, Z, cmap='viridis')
    # ax.set_xlabel('X-axis')
    # ax.set_ylabel('Y-axis')
    # ax.set_zlabel('Z-axis')
    # fig.colorbar(surf, ax=ax, shrink=0.5, aspect=10)

    # break
    # print(powerProfile)
    # powerProfileValues.append(powerProfile)

    # --- put into EKF
    # tracker.update_point_cloud(ranges, azimuths, elevations,dopplers, snrs, powerProfile)
    # targetDescr, tNum = tracker.step()
    # # frame_pcd = np.zeros((len(tracker.point_cloud),6))
    # frame_pcd = np.zeros((len(tracker.point_cloud),8))
    # for point_cloud, idx in zip(tracker.point_cloud, range(len(tracker.point_cloud))):
    #     frame_pcd[idx,0] = -np.sin(point_cloud.angle) * point_cloud.range
    #     frame_pcd[idx,1] = np.cos(point_cloud.angle) * point_cloud.range
    #     frame_pcd[idx,2] = np.cos(point_cloud.angle) * point_cloud.range
    #     frame_pcd[idx,3] = point_cloud.doppler 
    #     frame_pcd[idx,4] = point_cloud.snr
    #     frame_pcd[idx,5] = point_cloud.range
    #     frame_pcd[idx,6] = point_cloud.angle
    #     frame_pcd[idx,7] = point_cloud.power

        
#     pcd_datas.append(frame_pcd)
#     rawHeatmap.append(heatmap_log)
# pointcloud = np.array(pcd_datas)


In [13]:
from PIL import Image
import os

# Path to the folder containing images
output_gif = "pcd444.gif"

# Get list of image files (sorted by filename)
image_files = sorted([f for f in os.listdir("./visualization/pcd/") if f.endswith(('.png', '.jpg', '.jpeg'))])

# Load images
images = [Image.open(os.path.join("./visualization/pcd/", img)) for img in image_files]
images = images[-46:]
# Save as GIF
images[0].save(
    output_gif,
    save_all=True,
    append_images=images[1:],  # Append the rest of the images
    duration=.001,  # Duration per frame in milliseconds (adjust as needed)
    loop=0  # Loop indefinitely
)

print(f"GIF saved as {output_gif}")

GIF saved as pcd444.gif


In [ ]:
import pandas as pd


In [ ]:
np.save("./heatmap_eleRes.npy",heatmap_eleRes)
np.save("./heatmap_aziRes.npy",heatmap_aziRes)

In [ ]:
i=0
for frame in heatmap_aziRes:
    plt.imshow(np.abs(np.squeeze(frame,axis=0)))
    plt.savefig(f"./visualization/azi/{i}.png")
    i+=1

i=0
for frame in heatmap_eleRes:
    plt.imshow(np.abs(np.squeeze(frame,axis=0)))
    plt.savefig(f"./visualization/ele/{i}.png")
    i+=1

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os


for i, (azi_frame, ele_frame) in enumerate(zip(heatmap_aziRes, heatmap_eleRes)):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 12))
    azi_data = np.abs(np.squeeze(azi_frame, axis=0))
    ele_data = np.abs(np.squeeze(ele_frame, axis=0))
    vmin = min(azi_data.min(), ele_data.min())
    vmax = max(azi_data.max(), ele_data.max())
    im1 = ax1.imshow(azi_data, vmin=vmin, vmax=vmax, cmap='viridis')
    ax1.set_title('Azimuth Heatmap')
    im2 = ax2.imshow(ele_data, vmin=vmin, vmax=vmax, cmap='viridis')
    ax2.set_title('Elevation Heatmap')
    fig.suptitle(f'Radar Heatmap Analysis - Frame {i}', fontsize=14)
    plt.tight_layout(rect=[0, 0, 1, 0.95])
    plt.savefig(f"./visualization/merged/{i}.png")
    plt.close(fig)



In [ ]:
plt.imshow(np.abs(np.squeeze(heatmap_ele,axis=0)))

In [ ]:

#     elevationEst = np.fft.fft(elevationFFTInput,axis=0)
#     elevationEst=np.argmax(elevationEst)
#     elevationEst[elevationEst[:]>= LOOPS_PER_FRAME/2] -=  LOOPS_PER_FRAME
#     elevationFFTInput = np.einsum('ijk,jk->ik', elevationFFTInput, beamWeightsElevation)
#     # if not elevationFFTInput.shape[-1]:
#     #     continue
# for adc_data in all_data:
#     time_current = start_time_obj+timedelta(seconds=frameID*(200)/1000)
#     time_frames.append(time_current.strftime('%Y-%m-%d %H_%M_%S.%f'))
#     frameID+=1
#     radar_cube = dsp.range_processing(adc_data)
#     print("radar_cube.shape", radar_cube.shape)
#     mean = radar_cube.mean(0)
#     print("mean:",mean)
#     radar_cube = radar_cube - mean 
#     print("radar_cube.shape", radar_cube.shape)
#     rangeResult = radar_cube.sum(axis=(0,1))
#     # rangeResult.append(radar_cube)
#     ranges = np.arange(ADC_SAMPLES) * RANGE_RESOLUTION

#     plt.plot(ranges,np.abs(rangeResult))
#     plt.xlabel('Range (meters)')
#     plt.ylabel('Reflected Power')
#     plt.title('Interpreting a Single Chirp')
#     plt.show()
#     separate_tx_radar_cube = separate_tx(radar_cube,3,1,0)
#     doppler_fft_value = np.fft.fft(separate_tx_radar_cube, axis=0)
#     doppler_fft_value = np.fft.fftshift(doppler_fft_value, axes=0)
#     doppler_fft_value = np.abs(doppler_fft_value.sum(axis=1))
#     doppler_fft_value_scaled = (doppler_fft_value-np.min(doppler_fft_value))/(np.max(doppler_fft_value)-np.min(doppler_fft_value))
#     dopplerResult.append(doppler_fft_value_scaled)
#     # --- capon beamforming
#     beamWeights   = np.zeros(( VIRT_ANT,  ADC_SAMPLES), dtype=np.complex128)
#     radar_cube = np.concatenate((radar_cube[0::3,...], radar_cube[1::3,...], radar_cube[2::3,...]), axis=1)
#     # Note that when replacing with generic doppler estimation functions, radarCube is interleaved and
#     # has doppler at the last dimension.
#     # --- capon beamforming for 2D AOA
#     beamWeights_azimuth = np.zeros((VIRT_ANT, ADC_SAMPLES), dtype=np.complex_)
#     beamWeights_elevation = np.zeros((VIRT_ANT, ADC_SAMPLES), dtype=np.complex_)
#     range_azimuth_elevation = np.zeros((ANGLE_BINS, ANGLE_BINS, ADC_SAMPLES))

#     for i in range(ADC_SAMPLES):
#         range_azimuth_elevation[:,:,i], beamWeights_azimuth[:,i], beamWeights_elevation[:,i] = aoa_capon_2d(radar_cube[:, :, i].T, steering_vec_azimuth, steering_vec_elevation, magnitude=True)

#     # num_vec_azimuth, steering_vec_azimuth = gen_steering_vec_2d(ANGLE_RANGE, ANGLE_RES, NUM_RX * NUM_TX)
#     # num_vec_elevation, steering_vec_elevation = gen_steering_vec_2d(ANGLE_RANGE, ANGLE_RES, NUM_RX * NUM_TX)

#     # For each range bin
#     for i in range(ADC_SAMPLES):
#         range_azimuth_elevation[:,:,i], beamWeights_azimuth[:,i], beamWeights_elevation[:,i] = aoa_capon_2d(
#             radar_cube[:, :, i].T, steering_vec_azimuth, steering_vec_elevation, magnitude=True)
#         # --- cfar in azimuth, elevation, and range directions
#     first_pass, _ = np.apply_along_axis(func1d=dsp.ca_,
#                                         axis=0,
#                                         arr=np.log2(range_azimuth_elevation),
#                                         l_bound=1.5,
#                                         guard_len=4,
#                                         noise_len=16)

#     second_pass, noise_floor = np.apply_along_axis(func1d=dsp.ca_,
#                                                 axis=1,
#                                                 arr=np.log2(range_azimuth_elevation).transpose(1, 0, 2),
#                                                 l_bound=2.5,
#                                                 guard_len=4,
#                                                 noise_len=16)

#     third_pass, _ = np.apply_along_axis(func1d=dsp.ca_,
#                                         axis=2,
#                                         arr=np.log2(range_azimuth_elevation).transpose(2, 0, 1),
#                                         l_bound=2.5,
#                                         guard_len=4,
#                                         noise_len=16)

#     # --- classify peaks and calculate snrs
#     noise_floor = noise_floor.transpose(1, 0, 2)
#     first_pass = (np.log2(range_azimuth_elevation) > first_pass)
#     second_pass = (np.log2(range_azimuth_elevation).transpose(1, 0, 2) > second_pass).transpose(1, 0, 2)
#     third_pass = (np.log2(range_azimuth_elevation).transpose(2, 0, 1) > third_pass).transpose(1, 2, 0)
#     peaks = (first_pass & second_pass & third_pass)
#     peaks[:SKIP_SIZE, :, :] = 0
#     peaks[-SKIP_SIZE:, :, :] = 0
#     peaks[:, :SKIP_SIZE, :] = 0
#     peaks[:, -SKIP_SIZE:, :] = 0
#     peaks[:, :, :SKIP_SIZE] = 0
#     peaks[:, :, -SKIP_SIZE:] = 0
#     pairs = np.argwhere(peaks)
#     azimuths, elevations, ranges = pairs.T
#     snrs = np.log2(range_azimuth_elevation[pairs[:,0], pairs[:,1], pairs[:,2]]) - noise_floor[pairs[:,0], pairs[:,1], pairs[:,2]]

#     """ 4 (Doppler Estimation) """

#     # --- get peak indices
#     # beamWeights should be selected based on the range indices from CFAR.
#     dopplerFFTInput = radar_cube[:, :, ranges]
#     beamWeights  = beamWeights[:, ranges]

#     powerProfile = np.sum(np.abs(radar_cube) ** 2, axis=(0, 1))  
#     powerProfile  = powerProfile[ranges]

#     dopplerFFTInput = np.einsum('ijk,jk->ik', dopplerFFTInput, beamWeights)
#     if not dopplerFFTInput.shape[-1]:
#         continue
#     dopplerEst = np.fft.fft(dopplerFFTInput, axis=0)
#     dopplerEst = np.argmax(dopplerEst, axis=0)
#     dopplerEst[dopplerEst[:]>= LOOPS_PER_FRAME/2] -=  LOOPS_PER_FRAME
#     # --- convert bins to units
#     ranges = ranges * RANGE_RESOLUTION
#     azimuths = (azimuths - (ANGLE_BINS // 2)) * (np.pi / 180)
#     elevations = (elevations - (ANGLE_BINS // 2)) * (np.pi / 180)
#     dopplers = dopplerEst * DOPPLER_RESOLUTION
#     snrs = snrs


In [ ]:


# for adc_data in all_data:
#     time_current = start_time_obj+timedelta(seconds=frameID*(200)/1000)
#     time_frames.append(time_current.strftime('%Y-%m-%d %H_%M_%S.%f'))
#     frameID+=1
#     radar_cube = dsp.range_processing(adc_data)
#     print("radar_cube.shape", radar_cube.shape)
#     mean = radar_cube.mean(0)
#     print("mean:",mean)
#     radar_cube = radar_cube - mean 
#     print("radar_cube.shape", radar_cube.shape)
#     rangeResult = radar_cube.sum(axis=(0,1))
#     # rangeResult.append(radar_cube)
#     ranges = np.arange(ADC_SAMPLES) * RANGE_RESOLUTION

#     plt.plot(ranges,np.abs(rangeResult))
#     plt.xlabel('Range (meters)')
#     plt.ylabel('Reflected Power')
#     plt.title('Interpreting a Single Chirp')
#     plt.show()
#     separate_tx_radar_cube = separate_tx(radar_cube,3,1,0)
#     doppler_fft_value = np.fft.fft(separate_tx_radar_cube, axis=0)
#     doppler_fft_value = np.fft.fftshift(doppler_fft_value, axes=0)
#     doppler_fft_value = np.abs(doppler_fft_value.sum(axis=1))
#     doppler_fft_value_scaled = (doppler_fft_value-np.min(doppler_fft_value))/(np.max(doppler_fft_value)-np.min(doppler_fft_value))
#     dopplerResult.append(doppler_fft_value_scaled)
#     # --- capon beamforming
#     beamWeights   = np.zeros(( VIRT_ANT,  ADC_SAMPLES), dtype=np.complex128)
#     radar_cube = np.concatenate((radar_cube[0::3,...], radar_cube[1::3,...], radar_cube[2::3,...]), axis=1)
#     # Note that when replacing with generic doppler estimation functions, radarCube is interleaved and
#     # has doppler at the last dimension.
#     for i in range( ADC_SAMPLES):
#         range_azimuth[:,i], beamWeights[:,i] = dsp.aoa_capon(radar_cube[:, :, i].T, steering_vec, magnitude=True)
    
#     """ 3 (Object Detection) """
#     heatmap_log = np.log2(range_azimuth)
    
#     # --- cfar in azimuth direction
#     first_pass, _ = np.apply_along_axis(func1d=dsp.ca_,
#                                         axis=0,
#                                         arr=heatmap_log,
#                                         l_bound=1.5,
#                                         guard_len=4,
#                                         noise_len=16)
    
#     # --- cfar in range direction
#     second_pass, noise_floor = np.apply_along_axis(func1d=dsp.ca_,
#                                                 axis=0,
#                                                 arr=heatmap_log.T,
#                                                 l_bound=2.5,
#                                                 guard_len=4,
#                                                 noise_len=16)

#     # --- classify peaks and caclulate snrs
#     noise_floor = noise_floor.T
#     first_pass = (heatmap_log > first_pass)
#     second_pass = (heatmap_log > second_pass.T)
#     peaks = (first_pass & second_pass)
#     peaks[: SKIP_SIZE, :] = 0
#     peaks[- SKIP_SIZE:, :] = 0
#     peaks[:, : SKIP_SIZE] = 0
#     peaks[:, -SKIP_SIZE:] = 0
#     pairs = np.argwhere(peaks)
#     azimuths, ranges = pairs.T
#     snrs = heatmap_log[pairs[:,0], pairs[:,1]] - noise_floor[pairs[:,0], pairs[:,1]]

#     """ 4 (Doppler Estimation) """

#     # --- get peak indices
#     # beamWeights should be selected based on the range indices from CFAR.
#     dopplerFFTInput = radar_cube[:, :, ranges]
#     beamWeights  = beamWeights[:, ranges]

#     powerProfile = np.sum(np.abs(radar_cube) ** 2, axis=(0, 1))  
#     powerProfile  = powerProfile[ranges]

#     # --- estimate doppler values
#     # For each detected object and for each chirp combine the signals from 4 Rx, i.e.
#     # For each detected object, matmul (numChirpsPerFrame, numRxAnt) with (numRxAnt) to (numChirpsPerFrame)
#     dopplerFFTInput = np.einsum('ijk,jk->ik', dopplerFFTInput, beamWeights)
#     if not dopplerFFTInput.shape[-1]:
#         continue
#     dopplerEst = np.fft.fft(dopplerFFTInput, axis=0)
#     dopplerEst = np.argmax(dopplerEst, axis=0)
#     dopplerEst[dopplerEst[:]>= LOOPS_PER_FRAME/2] -=  LOOPS_PER_FRAME
    
#     """ 5 (Extended Kalman Filter) """

#     # --- convert bins to units
#     ranges = ranges *  RANGE_RESOLUTION
#     azimuths = (azimuths - ( NUM_ANGLE_BINS // 2)) * (np.pi / 180)
#     dopplers = dopplerEst *  DOPPLER_RESOLUTION
#     snrs = snrs
#     rangeAzimuthzResult.append([ranges,azimuths])
#     # print(f"ranges: {ranges.shape}, azimuths: {azimuths.shape}, dopplers: {dopplers.shape}, snrs: {snrs.shape}, powerProfile: {powerProfile.shape}")
    
#     # print(powerProfile)
#     # powerProfileValues.append(powerProfile)

#     # --- put into EKF
#     tracker.update_point_cloud(ranges, azimuths, dopplers, snrs, powerProfile)
#     targetDescr, tNum = tracker.step()
#     # frame_pcd = np.zeros((len(tracker.point_cloud),6))
#     frame_pcd = np.zeros((len(tracker.point_cloud),7))
#     for point_cloud, idx in zip(tracker.point_cloud, range(len(tracker.point_cloud))):
#         frame_pcd[idx,0] = -np.sin(point_cloud.angle) * point_cloud.range
#         frame_pcd[idx,1] = np.cos(point_cloud.angle) * point_cloud.range
#         frame_pcd[idx,2] = point_cloud.doppler 
#         frame_pcd[idx,3] = point_cloud.snr
#         frame_pcd[idx,4] = point_cloud.range
#         frame_pcd[idx,5] = point_cloud.angle
#         frame_pcd[idx,6] = point_cloud.power

        
#     pcd_datas.append(frame_pcd)
#     rawHeatmap.append(heatmap_log)
# pointcloud = np.array(pcd_datas)


In [ ]:
# import numpy as np
# import matplotlib.pyplot as plt
# from mpl_toolkits.mplot3d import Axes3D

# fig = plt.figure()
# ax = fig.add_subplot(111, projection='3d')

# # Assuming radar_cube is your data
# x, y, z = np.where(radar_cube != 0)  # Only plot non-zero values for clarity
# ax.scatter(x, y, z, c=radar_cube[x, y, z], cmap='viridis')

# ax.set_xlabel('Chirp Number')
# ax.set_ylabel('Antenna Number')
# ax.set_zlabel('Data Sample Value')
# plt.show()

In [ ]:
# import matplotlib.pyplot as plt

# # Plot a slice for one antenna
# plt.imshow(radar_cube[:, 0, :], aspect='auto', cmap='viridis')
# plt.colorbar(label='Signal Strength')
# plt.xlabel('Data Sample')
# plt.ylabel('Chirp Number')
# plt.title('Radar Data for Antenna 0')
# plt.show()

In [ ]:
# def gen_steering_vec_2d(azimuth_range, elevation_range, azimuth_res, elevation_res, num_tx, num_rx):
#     num_azimuth_bins = int(azimuth_range * 2 // azimuth_res + 1)
#     num_elevation_bins = int(elevation_range * 2 // elevation_res + 1)
    
#     # Antenna positions in meters (assuming a simple L-shape for TX antennas)
#     tx_positions = np.array([[0, 0, 0], [0.05, 0, 0], [0, 0.05, 0]])  # Left, Right, Upper
#     rx_positions = np.array([[0, 0, 0], [0.05, 0, 0], [0.1, 0, 0], [0.15, 0, 0]])  # Assuming linear array for RX
    
#     # Wavelength calculation
#     wavelength = 3e8 / (START_FREQ * 1e9)
    
#     # Generate steering vectors
#     steering_vec_azimuth = np.zeros((num_azimuth_bins, num_tx * num_rx), dtype=np.complex_)
#     steering_vec_elevation = np.zeros((num_elevation_bins, num_tx * num_rx), dtype=np.complex_)
    
#     for az_idx in range(num_azimuth_bins):
#         azimuth = (az_idx - num_azimuth_bins // 2) * np.pi / 180
#         for el_idx in range(num_elevation_bins):
#             elevation = (el_idx - num_elevation_bins // 2) * np.pi / 180
            
#             for tx_idx in range(num_tx):
#                 for rx_idx in range(num_rx):
#                     # Calculate phase shift for each antenna pair
#                     phase_shift = np.exp(-1j * 2 * np.pi / wavelength * 
#                                          (np.sin(elevation) * tx_positions[tx_idx, 2] + 
#                                           np.sin(azimuth) * np.cos(elevation) * (tx_positions[tx_idx, 0] + rx_positions[rx_idx, 0])))
#                     steering_vec_azimuth[az_idx, tx_idx * num_rx + rx_idx] = phase_shift
#                     steering_vec_elevation[el_idx, tx_idx * num_rx + rx_idx] = phase_shift
    
#     return steering_vec_azimuth, steering_vec_elevation

# # Generate steering vectors
# steering_vec_azimuth, steering_vec_elevation = gen_steering_vec_2d(ANGLE_RANGE, ANGLE_RANGE, ANGLE_RES, ANGLE_RES, NUM_TX, NUM_RX)


# def aoa_capon_2d(x, steering_vec_azimuth, steering_vec_elevation, magnitude=False):
#     """
#     Perform 2D AOA estimation using Capon (MVDR) Beamforming on a rx by chirp slice for both azimuth and elevation.

#     This function calculates the 2D angle spectrum via Capon beamforming method using one full frame as input.
#     It should be performed for each range bin to achieve AOA estimation for a full frame in both azimuth and elevation planes.

#     Args:
#         x (ndarray): Output of the 1D range fft with shape (num_ant, numChirps)
#         steering_vec_azimuth (ndarray): A 2D-array of size (numThetaAzimuth, num_ant) for azimuth estimation
#         steering_vec_elevation (ndarray): A 2D-array of size (numThetaElevation, num_ant) for elevation estimation
#         magnitude (bool): If True, return the magnitude of the spectrum, otherwise return complex values. Default=False

#     Raises:
#         ValueError: If steering vectors or input data are not the correct shape for matrix multiplication.

#     Returns:
#         A tuple containing:
#         - den_azimuth (ndarray): A 2D-Array of size (numThetaAzimuth, numThetaElevation) containing azimuth angle estimations for the given range
#         - den_elevation (ndarray): A 2D-Array of size (numThetaAzimuth, numThetaElevation) containing elevation angle estimations for the given range
#         - weights_azimuth (ndarray): A 2D-Array of size (num_ant, numThetaAzimuth) containing the Capon weights for azimuth
#         - weights_elevation (ndarray): A 2D-Array of size (num_ant, numThetaElevation) containing the Capon weights for elevation
#     """

#     if steering_vec_azimuth.shape[1] != x.shape[0] or steering_vec_elevation.shape[1] != x.shape[0]:
#         raise ValueError("Steering vectors and input data dimensions do not match for matrix multiplication.")

#     # Calculate the spatial covariance matrix Rxx
#     Rxx = cov_matrix(x)
#     Rxx = forward_backward_avg(Rxx)
#     Rxx_inv = np.linalg.inv(Rxx)

#     # Calculate for azimuth
#     first_azimuth = Rxx_inv @ steering_vec_azimuth.T
#     den_azimuth = np.reciprocal(np.einsum('ij,ij->i', steering_vec_azimuth.conj(), first_azimuth.T))
#     weights_azimuth = np.matmul(first_azimuth, den_azimuth)

#     # Calculate for elevation
#     first_elevation = Rxx_inv @ steering_vec_elevation.T
#     den_elevation = np.reciprocal(np.einsum('ij,ij->i', steering_vec_elevation.conj(), first_elevation.T))
#     weights_elevation = np.matmul(first_elevation, den_elevation)

#     # Create 2D spectrum
#     den_2d = np.outer(den_azimuth, den_elevation)

#     if magnitude:
#         return np.abs(den_2d), weights_azimuth, weights_elevation
#     else:
#         return den_2d, weights_azimuth, weights_elevation
